In [2]:
import pandas as pd # read in packages
import geopandas as gpd
import pyhhmm.gaussian as ga
import pyhhmm.utils as ut
import numpy as np

In [3]:
df = pd.read_csv('PinPoint 799 2021-10-08 10-04-41_activity.csv') # Read in the accelerometer data
gdf = gpd.read_file('799_cl.shp') # Read in the GPS data
outfile = '799_hmm.shp' # set the outfile 

In [4]:
df

,GMT Time,ODBA,Temperature [C],MT Time,Classification
0,8/10/2021 14:12,347,18.0,8/10/2021 8:12,NaN
1,8/10/2021 14:13,244,18.0,8/10/2021 8:13,NaN
2,8/10/2021 14:13,270,18.0,8/10/2021 8:13,NaN
3,8/10/2021 14:13,278,18.0,8/10/2021 8:13,NaN
4,8/10/2021 14:13,398,18.0,8/10/2021 8:13,NaN
...,...,...,...,...,...
340274,10/8/2021 16:01,291,15.0,10/8/2021 10:01,NaN
340275,10/8/2021 16:02,219,14.5,10/8/2021 10:02,NaN
340276,10/8/2021 16:02,386,14.5,10/8/2021 10:02,NaN
340277,10/8/2021 16:02,370,14.5,10/8/2021 10:02,NaN


In [5]:
df['GMT Time'] = pd.to_datetime(df['GMT Time']) # set  the GMT time field to be datetime
df['MT Time'] = df['GMT Time'] - pd.Timedelta(hours = 6) # subtract 6 hours from GMT time to get MT time

In [6]:
def run_hmm(df, x_features, n): # Define a function to run the hmm and return important components
    X = df[x_features] # set the x variable(s)
    n_samples = X.shape[0] # get the number of samples from the number of rows in X
    XY = np.zeros((X.shape[0],len(x_features))) # create an empty array which is the shape of the number of features (columns) and samples (rows)
    for x in X.index: # cycle through the number of samples
        for y in range(len(x_features)): # cycle through each feature
            XY[x,y] = X.iloc[x,y] # set the value of the empty array to be the same as the matching row/column pair in the dataframe 
            # If you can use numpy arrays, it makes laborious computations run quicker
    
    ghmm = ga.GaussianHMM( # Initialize the HMM
        n_states = n, # n = the predicted number of states
        n_emissions = len(x_features), # the number of emissions is the number of x variables
        covariance_type='diagonal', # do some research into this if you would like to change, there are times it may be appropriate
        verbose = False # leave this as false generally
    )
    ghmm, log_likelihood = ghmm.train( # Train the HMM 
        [XY], # use the np.array with your x variable data
        n_init = 1, # generally leave these things the same
        n_iter = 1,
        conv_thresh = 0.01,
        plot_log_likelihood=False,
        no_init=False
    )
    return(ghmm) # Return the trained HMM
res = run_hmm(df,x_features=['ODBA'],n = 2) # Run this using your accelerometer data   


In [7]:
print(res.init_params)# print out some values from the HMM

stmc


In [8]:
print(res)

Pi: [0.57075696 0.42924304]
A:
[[0.75369897 0.24630103]
 [0.23251366 0.76748634]]
Means:
[[ 23.42329517]
 [162.46584111]]
Covariances:
[[[  876.94167222]]

 [[17520.93197949]]]


In [9]:
gdf.columns

Index(['GMT Time', 'Latitude', 'Longitude', 'Altitude', 'Duration',
       'Temperatur', 'Voltage', 'DOP', 'Satellites', 'Cause of F', 'Lab_Coord',
       'Lab_Dur', 'Lab_Sat', 'Lab_Alt', 'No', 'time', 'distance', 'speed',
       'Dist_Pt1', 'Dist_Pt2', 'Dist_Pt3', 'Status', 'FM_R', 'FM_G', 'FM_T',
       'geometry'],
      dtype='object')

In [10]:
# Set gdf time to be MT time
gdf['GMT Time'] = pd.to_datetime(gdf['GMT Time'])
gdf['MT Time'] = gdf['GMT Time'] - pd.Timedelta(hours=6)
gdf

,GMT Time,Latitude,Longitude,Altitude,Duration,Temperatur,Voltage,DOP,Satellites,Cause of F,...,speed,Dist_Pt1,Dist_Pt2,Dist_Pt3,Status,FM_R,FM_G,FM_T,geometry,MT Time
0,2021-08-10 15:00:00,36.438997,-109.090688,2672.21,33,18.0,0,1.2,6,GPS Schedule,...,-1.000000,2277.6300,3964.4000,13079.7000,2277.6300,0.300000,1.000000,0.300000,POINT (671132.417 4034335.408),2021-08-10 09:00:00
1,2021-08-10 15:15:00,36.444590,-109.104966,2696.63,24,18.0,0,1.8,5,GPS Schedule,...,63.744261,1117.8900,3368.7400,12906.9000,1117.8900,0.000000,0.300000,1.000000,POINT (669840.277 4034930.662),2021-08-10 09:15:00
2,2021-08-10 15:30:00,36.454565,-109.107341,2727.37,2,18.0,0,1.8,5,GPS Schedule,...,28.709792,10.4177,2368.0300,11976.2000,10.4177,0.302734,0.001172,0.298828,POINT (669605.699 4036033.095),2021-08-10 09:30:00
3,2021-08-10 15:45:00,36.454381,-109.107407,2721.39,2,19.5,0,2.0,5,GPS Schedule,...,1.033541,10.8616,2389.1800,11997.2000,10.8616,1.000000,0.300000,0.000000,POINT (669600.185 4036012.544),2021-08-10 09:45:00
4,2021-08-10 16:00:00,36.454364,-109.107184,2727.23,2,21.0,0,2.4,5,GPS Schedule,...,0.831016,21.0219,2383.4000,11990.5000,21.0219,1.000000,0.300000,0.000000,POINT (669620.197 4036011.116),2021-08-10 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,2021-10-08 14:30:00,36.552433,-109.050749,1846.72,17,20.5,0,1.6,7,GPS Schedule,...,26.364314,11995.3000,9620.4000,16.8758,16.8758,0.321858,0.009368,0.290632,POINT (674457.762 4046991.987),2021-10-08 08:30:00
3076,2021-10-08 15:00:00,36.474546,-109.097644,2461.73,16,17.5,0,4.0,5,GPS Schedule,...,26.280874,2391.7900,37.1275,9600.4400,37.1275,0.322881,0.009806,0.290194,POINT (670431.042 4038266.901),2021-10-08 09:00:00
3077,2021-10-08 15:15:00,36.474649,-109.097595,2463.06,15,17.5,0,3.4,4,GPS Schedule,...,0.667088,2404.0400,43.2567,9588.2400,43.2567,1.000000,0.300000,0.000000,POINT (670435.170 4038278.436),2021-10-08 09:15:00
3078,2021-10-08 15:46:00,36.474399,-109.097615,2424.88,62,18.5,0,3.8,5,GPS Schedule,...,3.920703,2377.6000,26.1756,9613.9500,26.1756,1.000000,0.300000,0.000000,POINT (670433.971 4038250.688),2021-10-08 09:46:00


In [11]:
def pred_hmm(GPS,acc,hmm):# Define a function to predict values
    r_dict = {} # create a result dictionary
    my_reslist = [] # create a result list
    run = 0 # create a running counter
    time_list = GPS['MT Time'].to_list() # create a list of the MT time values (in your GPS points)
    for x in time_list: # run through all of the times
        print(run,x) 
        time_1 = x - pd.Timedelta(minutes = 7, seconds = 30) # get a time range, this should be half the time of hte GPS interval (15/2 = 7.5)
        time_2 = x + pd.Timedelta(minutes = 7, seconds = 30)
        mask = (acc['MT Time'] >= time_1) & (acc['MT Time'] <= time_2) # create a mask to select only time points within your window in the accelerometer data
        data = acc.loc[mask, ['MT Time','ODBA']] # select only the ODBA data
        mydata = np.zeros((data.shape[0], 1)) # create a np.array that is the same shape as your data with a single emission field
        for x in range(data.shape[0]): # set the data at each row/column location in your array to the corresponding value in the dataframe
            mydata[x,0] = data.iloc[x,1]
        results = hmm.predict([mydata]) # Results then equal the predicted values at each point
        r_dict[run] = results # set the dictionary value at the run location to the dictionary of results
        print(results)
        acc.loc[mask,'U_HMM'] = results[0] # 
        run +=1
        for r in range(len(results[0])): # append each result dictionary to the result list 
            my_reslist.append(results[0][r])
            print(results[0][r])
                
    return(r_dict,my_reslist)
results, list_r = pred_hmm(gdf,df,res)

0 2021-08-10 09:00:00
[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0])]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
1 2021-08-10 09:15:00
[array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])]
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
2 2021-08-10 09:30:00
[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
1
1
1


Examine the structure of your result and list_r objects, they will be important in later steps 

In [12]:
print(list_r)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
df.loc[:,'U_HMM'] = df.loc[:,'U_HMM'].astype('int') # set the value of U_HMM to integer

C:\Users\chris\AppData\Local\Temp/ipykernel_38492/3768243616.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['U_HMM'] = df['U_HMM'].astype('int')


In [ ]:
from sklearn.metrics import classification_report # import classification report
res = classification_report(df['Classification'],df['U_HMM'], labels = [0,1], output_dict=True) # run the classificaiton report with the labels and predicted classifications

res = pd.DataFrame(res)
res

c:\ProgramData\Anaconda3\envs\geopandas_docs\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\envs\geopandas_docs\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\envs\geopandas_docs\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

,0,1,accuracy,macro avg,weighted avg
precision,0.0,0.830508,0.830508,0.415254,0.689744
recall,0.0,1.000000,0.830508,0.500000,0.830508
f1-score,0.0,0.907407,0.830508,0.453704,0.753610
support,20.0,98.000000,0.830508,118.000000,118.000000


In [16]:
onesdict = {}  # create a dictionary to hold all of the one results
zerosdict = {} # create a dictionary to hold all of the zero results 
run = 0 # create a running calculator
for x in results: # cycle throug the HMM results
    den = len(results[x][0]) # get the length of the results dictionary in question to determine the denominator of your classification rate formula
    ones = 0  # set a running counter
    zeros = 0 # set a running counter
    for y in results[x][0]: # run through everything in the results dictionary
        if y ==1: # set ones to be one more if the classification was one and the same for zero
            ones += 1
        else:
            zeros += 1
    onesrate = ones / den # ones counter/divided by total length = classificastion rate
    zerorate = zeros / den
    onesdict[run] = onesrate # set the onesdict value at the run counter to the onesrate, and the same for zerosdict
    zerosdict[run] = zerorate
    run += 1
zerosdict

{0: 0.9333333333333333,
 1: 0.1,
 2: 0.13333333333333333,
 3: 0.8833333333333333,
 4: 0.9,
 5: 0.9666666666666667,
 6: 0.25,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.8666666666666667,
 16: 0.95,
 17: 0.8666666666666667,
 18: 0.6,
 19: 0.9333333333333333,
 20: 1.0,
 21: 0.9166666666666666,
 22: 0.9166666666666666,
 23: 0.8666666666666667,
 24: 0.8166666666666667,
 25: 0.95,
 26: 0.9833333333333333,
 27: 1.0,
 28: 0.9333333333333333,
 29: 0.95,
 30: 0.9,
 31: 0.8166666666666667,
 32: 0.8833333333333333,
 33: 0.9166666666666666,
 34: 0.95,
 35: 0.9166666666666666,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.95,
 48: 0.9833333333333333,
 49: 0.9166666666666666,
 50: 0.9833333333333333,
 51: 0.9666666666666667,
 52: 1.0,
 53: 0.8833333333333333,
 54: 0.9,
 55: 0.8833333333333333,
 56: 0.9166666666666666,
 57: 0.9666666666666667,
 58: 0.7666666666666667,
 59: 1.0,
 60: 0.966666

In [17]:
gdf['ones'] = onesdict.values() # set the gdf value at each GPS point to be the CR rate and set it to be float type
gdf['zeros'] = zerosdict.values()
gdf['ones'] = gdf['ones'].astype('float')
gdf['zeros'] = gdf['zeros'].astype('float')
gdf

,GMT Time,Latitude,Longitude,Altitude,Duration,Temperatur,Voltage,DOP,Satellites,Cause of F,...,Dist_Pt2,Dist_Pt3,Status,FM_R,FM_G,FM_T,geometry,MT Time,ones,zeros
0,2021-08-10 15:00:00,36.438997,-109.090688,2672.21,33,18.0,0,1.2,6,GPS Schedule,...,3964.4000,13079.7000,2277.6300,0.300000,1.000000,0.300000,POINT (671132.417 4034335.408),2021-08-10 09:00:00,0.066667,0.933333
1,2021-08-10 15:15:00,36.444590,-109.104966,2696.63,24,18.0,0,1.8,5,GPS Schedule,...,3368.7400,12906.9000,1117.8900,0.000000,0.300000,1.000000,POINT (669840.277 4034930.662),2021-08-10 09:15:00,0.900000,0.100000
2,2021-08-10 15:30:00,36.454565,-109.107341,2727.37,2,18.0,0,1.8,5,GPS Schedule,...,2368.0300,11976.2000,10.4177,0.302734,0.001172,0.298828,POINT (669605.699 4036033.095),2021-08-10 09:30:00,0.866667,0.133333
3,2021-08-10 15:45:00,36.454381,-109.107407,2721.39,2,19.5,0,2.0,5,GPS Schedule,...,2389.1800,11997.2000,10.8616,1.000000,0.300000,0.000000,POINT (669600.185 4036012.544),2021-08-10 09:45:00,0.116667,0.883333
4,2021-08-10 16:00:00,36.454364,-109.107184,2727.23,2,21.0,0,2.4,5,GPS Schedule,...,2383.4000,11990.5000,21.0219,1.000000,0.300000,0.000000,POINT (669620.197 4036011.116),2021-08-10 10:00:00,0.100000,0.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,2021-10-08 14:30:00,36.552433,-109.050749,1846.72,17,20.5,0,1.6,7,GPS Schedule,...,9620.4000,16.8758,16.8758,0.321858,0.009368,0.290632,POINT (674457.762 4046991.987),2021-10-08 08:30:00,0.650000,0.350000
3076,2021-10-08 15:00:00,36.474546,-109.097644,2461.73,16,17.5,0,4.0,5,GPS Schedule,...,37.1275,9600.4400,37.1275,0.322881,0.009806,0.290194,POINT (670431.042 4038266.901),2021-10-08 09:00:00,0.066667,0.933333
3077,2021-10-08 15:15:00,36.474649,-109.097595,2463.06,15,17.5,0,3.4,4,GPS Schedule,...,43.2567,9588.2400,43.2567,1.000000,0.300000,0.000000,POINT (670435.170 4038278.436),2021-10-08 09:15:00,0.033333,0.966667
3078,2021-10-08 15:46:00,36.474399,-109.097615,2424.88,62,18.5,0,3.8,5,GPS Schedule,...,26.1756,9613.9500,26.1756,1.000000,0.300000,0.000000,POINT (670433.971 4038250.688),2021-10-08 09:46:00,0.216667,0.783333


In [18]:
gdf['GMT Time'] = gdf['GMT Time'].astype('str') # set GMT and MT time to string so you can save the file 
gdf['MT Time'] = gdf['MT Time'].astype('str')

In [19]:
gdf.to_file(outfile)